In [68]:
import matplotlib
matplotlib.use('pdf')

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
plt.style.use('default')

In [70]:
def human_format(x):
    sigpart = 0
    suffix = ""
    if x < 1000:
        return "{:.3g}".format(x)
    elif x < 1000000:
        sigpart = x / 1000
        suffix = "k"
    elif x < 1000000000:
        sigpart = x / 1000000
        suffix = "M"
    else:
        sigpart = x / 1000000000
        suffix = "B"
    return "{:.3g}{}".format(sigpart, suffix)
def get_algorithm(path):
    algs = ["rkde", "ic2", "sklearn", "nocut", "simple"]
    for alg in algs:
        if alg in path:
            return alg
def get_dataset(path):
    datasets = {
        "bgauss": "gauss",
        "hep": "hep",
        "tmy3_4": "tmy3_4",
        "tmy3.": "tmy3",
        "mnist": "mnist",
        "home": "home",
    }
    for dataset in datasets.keys():
        if dataset in path:
            return datasets[dataset]
markers = {
    "ic2": "o",
    "simple": "^",
    "sklearn": "s",
    "rkde": "d",
    "nocut": "8",
    "ks": "*",
}

In [74]:
df = pd.read_csv("../experiments/d_raiders/scale_d.csv")

In [75]:
df["r_algorithm"] = df["out_path"].map(get_algorithm)
df["r_dataset"] = df["out_path"].map(get_dataset)
df["throughput"] = (
    1000 * df["num_test"] / df["test_time"]
)
df["a_throughput"] = (
    1000 * df["num_train"] 
    / (df["test_time"]*df["num_train"]/df["num_test"] + df["train_time"])
)

In [76]:
df = df.sort_values(by=["r_algorithm", "dim"])

In [77]:
algorder = ["ic2", "simple", "sklearn", "rkde"]

In [83]:
matplotlib.rcParams.update({'font.size': 11})
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, LogLocator
fig, ax = plt.subplots(figsize=(6,3.5),dpi=300)
for i,alg in enumerate(algorder):
    cur_df = df[df["r_algorithm"] == alg]
    ax.loglog(
        cur_df["dim"],
        cur_df["throughput"],
        marker=markers[alg],
        linewidth=1.0,
        markersize=4.0,
        label=alg,
        basex=2,
    )
majorFormatter = FormatStrFormatter('%d')
ax.xaxis.set_major_formatter(majorFormatter)
# ax.xaxis.set_major_locator(LogLocator(base=2, subs=[1,2]))
ax.xaxis.set_ticks([1,2,4,8,16,32,64,128,256,784])
ax.set_xlabel("Dataset Dimension")
ax.set_ylabel("Queries / s")
ax.set_title("Adjusting Dimension, mnist, n={}".format(
        human_format(df.num_train.iloc[0]),
        df.dim.iloc[0]))
ax.legend(loc=0, ncol=2)
ax.yaxis.grid(True, ls="dotted", alpha=0.5)
fig.tight_layout()
fig.savefig("d_scale.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
